In [1]:
from custom_retina import *
import tensorflow as tf

padded = True # to generate the "padded" model, which fails edgetpu compilation
# if "False", the model compiles with edgetpucompiler

num_classes = 5
model = MobileRetinaNet(num_classes, conv_filters=16, num_convs = 3, padded = padded)

if padded:
    input_shape = (120, 160, 3)
else:
    # To remove the "Pad" operation, the input height must have 1 more pixel on the "Add" operation, so 4 pixels on input shape
    input_shape = (124, 160, 3) 

input_shape_with_none = (None,) + input_shape
print(input_shape_with_none)
model.build(input_shape = input_shape_with_none)

one_sample = tf.random.normal(shape = (1,) + input_shape )

y = model(one_sample)
print(y.shape)
print(y[0].shape)

model.summary()

/home/tensorflow/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


(None, 120, 160, 3)
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
shape p3_output        : (1, 15, 20, 16)
shape p3_output_padded : (1, 16, 20, 16)
shape p4_output : (1, 8, 10, 16)
shape self.upsample_2x(p4_output) : (1, 16, 20, 16)
(1, 3852, 9)
(3852, 9)
Model: "MobileRetinaNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 backbone-mobile-net-v2 (Fun  [(None, 15, 20, 192),    2257984   
 ctional)                     (None, 8, 10, 576),                
                              (None, 4, 5, 1280)]                
                                                                 
 FeaturePyramid (FeaturePyra  multiple                 2484416   
 mid)                                                            
                                                          

In [2]:
loss_fn = RetinaNetLoss(num_classes)
learning_rates = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)
optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss = loss_fn, optimizer=optimizer)

In [3]:
# Trying to fit it to zeros, just to test the architecture
x = tf.random.normal(shape = (100,) + input_shape )
true_y = tf.zeros(shape = (100, 3852, 9))

train_x = x[:80,...]
train_y = true_y[:80,...]
test_x = x[80:,...]
test_y = true_y[80:,...]

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

BATCH_SIZE = 7
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

model.fit(train_dataset,
    epochs=2,
    validation_data = test_dataset)

Epoch 1/2
shape p3_output        : (7, 15, 20, 16)
shape p3_output_padded : (7, 16, 20, 16)
shape p4_output : (7, 8, 10, 16)
shape self.upsample_2x(p4_output) : (7, 16, 20, 16)
shape p3_output        : (7, 15, 20, 16)
shape p3_output_padded : (7, 16, 20, 16)
shape p4_output : (7, 8, 10, 16)
shape self.upsample_2x(p4_output) : (7, 16, 20, 16)
11/11 [==============================] - ETA: 0s - loss: 1.1284shape p3_output        : (7, 15, 20, 16)
shape p3_output_padded : (7, 16, 20, 16)
shape p4_output : (7, 8, 10, 16)
shape self.upsample_2x(p4_output) : (7, 16, 20, 16)
11/11 [==============================] - 7s 174ms/step - loss: 1.1284 - val_loss: 1.1284
Epoch 2/2
11/11 [==============================] - 1s 95ms/step - loss: 1.1284 - val_loss: 1.1284


In [4]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os

def representative_dataset_gen():
    count = 0
    files = os.listdir("representative_dataset/")
    for j in files:
        if count<200:
            img = Image.open("representative_dataset/" + j)
            img = img.resize((160, 124))

            width, height = img.size
            bands = img.getbands()
            array = np.asarray(img, dtype=np.float32)
            #array = preprocess_input(array)
            count=count+1
            yield[np.expand_dims(array, axis=0)]
        else:
            break

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.experimental_new_converter = False  # This is needed for the converter to work (false)
# and with True, there are strange Pack / Reshape operations

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

filename = 'mobile_retina_net' + ("_padded" if padded else "_not_padded") + '.tflite'
with open(filename, 'wb') as f:
  f.write(tflite_model)


shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
get_config RetinaNetLoss
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 20, 16)
shape p4_output : (None, 8, 10, 16)
shape self.upsample_2x(p4_output) : (None, 16, 20, 16)
shape p3_output        : (None, 15, 20, 16)
shape p3_output_padded : (None, 16, 

INFO:tensorflow:Assets written to: /tmp/tmpsqmmzvjv/assets


INFO:tensorflow:Assets written to: /tmp/tmpsqmmzvjv/assets


shape p3_output        : (1, 15, 20, 16)
shape p3_output_padded : (1, 16, 20, 16)
shape p4_output : (1, 8, 10, 16)
shape self.upsample_2x(p4_output) : (1, 16, 20, 16)


/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Estimated count of arithmetic ops: 271.352 M  ops, equivalently 135.676 M  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [5]:
tf.__version__

'2.10.0'

array([[19, 22],
       [43, 50]])